# Ансамбли

1111


In [7]:
import sys

from tensorflow.keras import datasets, layers, models, losses
import numpy as np
from PIL import Image
import tensorflow as tf
import torchvision as tv
import matplotlib.pyplot as plt
from livelossplot import PlotLossesKeras
import tensorflow_addons as tfa

sys.path.append('../../../')

from core.datasets import get_ds
from core.make_answer import make_ans_file

In [11]:
transformer = tv.transforms.Compose([
    tv.transforms.RandomAdjustSharpness(sharpness_factor=5, p=.4),
    tv.transforms.RandomHorizontalFlip(p=.5),
    tv.transforms.RandomPosterize(bits=3, p=.3),
    tv.transforms.RandomPerspective(distortion_scale=0.2, p=.5),
    tv.transforms.RandomRotation(degrees=(-10, 10)),
    tv.transforms.ColorJitter(brightness=.2, hue=0.2),
])

In [12]:
train_ds, val_ds, test_ds = get_ds('repaired_data_train', 'repaired_data_test', transform=transformer, one_hot=True, back=3)

In [13]:
model_1 = tf.keras.models.load_model(f'../../../models/checkpoints/model1_categorical_accuracy.h5')
model_3 = tf.keras.models.load_model(f'../../../models/checkpoints/model3_categorical_accuracy.h5')

- Bagging (пакетирование) голосование, предпочтительна система наподобие земских соборов с представителями разных сословий нейронных сетей.

In [18]:
def get_bagging_pred(models, x):
    pred_summ = np.full((len(x), 10), 0.)
    for m in models:
        pred_summ += m.predict(x)
    pred_summ = np.array([np.argmax(i) for i in pred_summ])
    return pred_summ


batch = train_ds[0]
y_true = np.array([np.argmax(i) for i in batch[1]])
y_pred_ensemble = get_bagging_pred(models=[model_1, model_3], x=batch[0])
y_pred_model_1 = np.array([np.argmax(i) for i in model_1.predict(batch[0])])
y_pred_model_3 = np.array([np.argmax(i) for i in model_3.predict(batch[0])])

print("model_1", np.sum(y_true == y_pred_model_1) / 512)
print("model_3", np.sum(y_true == y_pred_model_3) / 512)
print("ensemble", np.sum(y_true == y_pred_ensemble) / 512)

# mmm = MulticlassClassificationMetrics(y_true=y_true, y_pred=y_pred)
# print(mmm.accuracy())

16/16 [==============================] - 0s 15ms/step
model_1 0.634765625
model_3 0.607421875
ensemble 0.646484375
